<a href="https://colab.research.google.com/github/cvs2010/rsp/blob/main/notebook_teste_importsheets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook criado com intuito de gerar um algoritmo para importação dos dados do número de operadores por hora das planilhas Horario RSP

## Observações:
- Tente o mínimo possível utilizar as bandas de GPU e TPU grátis do colab
- Esse arquivo é um arquivo de histórico, então tente trabalhar no seu GoogleColab em paralelo, somente suba as células para esse Notebook após finalizar todo o código ou gerar um checkpoint de análise do código (momento em que o código ainda não está pronto, mas necessita de análise do grupo)
- Muito cuidado com a edição desse arquivo e de outros arquivos da pasta do Drive: mais uma vez, tente trabalhar em paralelo, nunca utilize os arquivos originais, faça cópia de arquivos e trabalhe com seus arquivos localmente ou via cloud em seu login.

# Initial Configs
Configurações Iniciais

## Importação e Instalação de bibliotecas:
- bibliotecas disponíveis: import scipy, matplotlib, numpy, pandas, sklearn, seaborn, skimage, xgboost, lightgbm, catboost, tensorflow, keras, theano, pydot, graphviz, h5py, cv2, PIL, requests, bs4, lxml, html5lib, pyproj, shapely, geopandas, folium, nltk, spacy, textblob, gensim, glove_python, transformers, sentence_transformers, torchvision, torchtext, spacy, openai, tqdm, watermark, imageio, moviepy, simpleaudio, pygame, bokeh, plotly, dash, dash_core_components, dash_html_components, dash_bootstrap_components, ipywidgets, ipython, autoreload, pprint, inspect, warnings, logging, time, datetime, os, sys, math, random, json, re, hashlib, base64, uuid, collections, itertools, functools

In [2]:
#Importação de Bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import numpy as np
import os
import re
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelBinarizer
from tensorflow import keras
from datetime import datetime
import urllib.request
import zipfile
import io

#Instalação de Bibliotecas
!pip install PyPDF2
!pip install spacy && python -m spacy download pt_core_news_sm
!pip install keras
!pip install tabula-py
!pip install pdfplumber
!pip install unidecode
from unidecode import unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-05-04 13:47:10.951931: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 43.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 6.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 6.0 MB/s eta 0:00:00


Install 7zip reader libarchive

In [3]:
# https://pypi.python.org/pypi/libarchive
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
import libarchive

Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 122518 files and directories currently installed.)
Preparing to unpack .../libarchive-dev_3.4.0-2ubuntu1.2_amd64.deb ...
Unpacking libarchive-dev:amd64 (3.4.0-2ubuntu1.2) ...
Setting up libarchive-dev:amd64 (3.4.0-2ubuntu1.2) ...
Processing triggers for man-db (2.9.1-1) ...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 4.8 MB/s eta 0:00:00
  Created wheel for libarchive: filename=libarchive-0.4.7-py3-none-any.whl size=31644 sha256=42b2f26fa438ca456abcca90e12746a176dd27ad00124d4a96fb21f1cfffc847
  Stored in directory: /root/.cache/pip/wheels/3a/94/d0/6cd83c8a80a4236fd4cb2a1fd846ecf72ab1e0ac238c5951c0
Successfully built libarchive


## Montando GoogleDrive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Importação do DataFrame

In [5]:
df = pd.read_csv('https://github.com/cvs2010/rsp/raw/main/files/dataframe_backup.csv')

In [6]:
df

,horas,num_traf,dd_mm_aaaa,ds_fs_ff,00:00,00:15,00:30,00:45,01:00,01:15,...,21:30,21:45,22:00,22:15,22:30,22:45,23:00,23:15,23:30,23:45
0,D1Y1,1048,01/01/2017,DOM,4,2,3,2,2,2,...,6,2,9,4,9,8,4,4,4,6
1,D2Y1,962,02/01/2017,SEG,3,4,5,2,2,2,...,9,9,8,10,9,9,4,3,3,10
2,D3Y1,1150,03/01/2017,TER,2,4,3,2,2,2,...,10,8,4,7,7,5,3,3,2,7
3,D4Y1,685,04/01/2017,QUA,3,4,4,2,2,2,...,6,4,6,6,8,10,5,9,8,5
4,D5Y1,1017,05/01/2017,QUI,5,4,2,2,2,2,...,4,9,5,7,10,2,7,7,7,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1820,D361Y5,913,27/12/2023,QUA,2,3,3,2,2,2,...,10,5,3,10,5,3,6,3,8,5
1821,D362Y5,1082,28/12/2023,QUI,3,4,2,2,2,2,...,5,2,9,3,4,4,9,4,2,6
1822,D363Y5,1154,29/12/2023,SEX,5,5,4,2,2,2,...,10,4,7,6,6,7,6,2,6,6
1823,D364Y5,963,30/12/2023,SÁB,4,5,3,2,2,2,...,10,5,2,8,6,6,2,6,7,3


# Extração dos dados (linha de nº de operadores) de planilhas

## Importando DataSet
- Importando o DataSet HorarioTeste.zip do repositório do github
- Extraindo os arquivos na pasta /content/


In [7]:
# !wget https://github.com/cvs2010/rsp/releases/download/v0.0.1-alpha/horariosRSP_dataSet.zip
# !unzip horariosRSP_dataSet.zip -d /content/HorariosTeste

import urllib.request
import zipfile
import io

url = "https://github.com/cvs2010/rsp/releases/download/v0.0.1-alpha/horariosRSP_dataSet.zip"
with urllib.request.urlopen(url) as u:
    f = io.BytesIO(u.read())

with zipfile.ZipFile(f, 'r') as zip_ref:
    zip_ref.extractall('/content/HorariosTeste')

## Resgatando a informação de data do nome do arquivo
Após a extração da data e transformação em uma string de data legível, iremos inserir essa informação na linha/coluna de data do dataframe
- O próximo passo é fazer com que ele leia todos os arquivos dentro da pasta HorariosTeste e processar um a um automaticamente de forma ordenada

In [103]:
# Para fazer esse processo são necessárias as bibliotecas os, re, unidecode e datetime

# Caminho do arquivo
filename = '/content/HorariosTeste/2022_2023/Horário RSP - Backup - 12-04-2023 - WED.xlsx'

# Expressão regular para encontrar a data no nome do arquivo
date_regex = r'(\d{1,2})[-_./](\d{1,2})[-_./](\d{4})'

# Encontra a primeira data correspondente no nome do arquivo
match = re.search(date_regex, filename)

# Extrai a data no formato dd-mm-yyyy
def extract_date(filename):
    pattern = r'(\d{2})\s(\w{3})\s(\d{4})'
    match = re.search(pattern, filename)
    if match:
        day, month, year = match.groups()
        months = {
            'JAN': '01', 'FEB': '02', 'MAR': '03', 'APR': '04', 'MAY': '05', 'JUN': '06',
            'JUL': '07', 'AUG': '08', 'SEP': '09', 'OCT': '10', 'NOV': '11', 'DEC': '12'
        }
        month = months[month.upper()]
        return f'{day}/{month}/{year}'
    else:
        return ""

# Rodando a função filename
formatted_date = filename
date = extract_date(formatted_date)
print(filename)
print(date)

# Extrai a data no formato dd-mm-yyyy
if match:
    day = match.group(1).zfill(2)
    month = match.group(2).zfill(2)
    year = match.group(3)
    formatted_date = f'{day}/{month}/{year}'
    print(formatted_date)
else:
    print('')

/content/HorariosTeste/2022_2023/Horário RSP - Backup - 12-04-2023 - WED.xlsx

12/04/2023


## Extraindo a linha 'OPERADORES'

Extração de arquivos pdf

In [104]:
# !pip install tabula-py

from tabula import read_pdf

# Caminho para o arquivo PDF
file_path = '/content/HorariosTeste/2022_JAN_ABRIL/1. JANEIRO/MANH╟/01 JAN 2022 SAB MANH╟.pdf'

# Ler a tabela da página 1 do PDF
planilha1 = read_pdf(file_path, pages='1')

# Exibir o DataFrame do Pandas
planilha1

[    Unnamed: 0                                         Unnamed: 1  Unnamed: 2  \
 0          NaN                                                NaN         NaN   
 1          NaN                                                NaN         NaN   
 2          NaN                                              MANHÃ         NaN   
 3          NaN                                                NaN         0.0   
 4          NaN                                              NOMES         NaN   
 5          1.0                                              BRUNO         NaN   
 6          2.0                                           HAYANNEE         NaN   
 7          3.0                                    LETÍCIA BRANDÃO         NaN   
 8          4.0                                            LACERDA         NaN   
 9          5.0                                              ZARRO         NaN   
 10         6.0                                   TAMIRES CARVALHO         NaN   
 11         7.0 

Extração de arquivos de planilha

In [105]:
# Essa célula necessita da biblioteca pandas importada

# Caminho para o arquivo Excel
file_path = '/content/HorariosTeste/2017/01 JANEIRO/MANH╟/01 JAN 2017 DOM MANH╟.xls'


# Ler a planilha "MANHÃ" para um DataFrame (df) do Pandas
planilha1 = pd.read_excel(file_path)

# Mostrar as últimas 20 linhas da Planilha
planilha1.head(50)

,Unnamed: 0,Unnamed: 1,TURNO MANHÃ,Unnamed: 3,6,Unnamed: 5,Unnamed: 6,Unnamed: 7,7,Unnamed: 9,Unnamed: 10,Unnamed: 11,8,Unnamed: 13,Unnamed: 14,Unnamed: 15,9,Unnamed: 17,Unnamed: 18,Unnamed: 19,10,Unnamed: 21,Unnamed: 22,Unnamed: 23,11,Unnamed: 25,Unnamed: 26,Unnamed: 27,12,Unnamed: 29,Unnamed: 30,Unnamed: 31,13,Unnamed: 33,Unnamed: 34,Unnamed: 35,14,Unnamed: 37,Unnamed: 38,Unnamed: 39,Horas,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,30,45,0,15,30.1,45.1,0.1,15.1,30.2,45.2,0.2,15.2,30.3,45.3,0.3,15.3,30.4,45.4,0.4,15.4,30.5,45.5,0.5,15.5,30.6,45.6,0.6,15.6,30.7,45.7,0.7,15.7,30.8,45.8,0.8,15.8,30.9,45.9
0,NaN,NaN,NOME,NaN,0,15,30,45,0,15,30,45,0,15,30,45,0,15,30,45,0,15,30,45,0,15,30,45,0,15,30,45,0,15,30,45,0,15,30,45,Trabalhadas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,NaN,NaN,NaN,NaN,06:00:00,06:15:00,06:30:00,06:45:00,07:00:00,07:15:00,07:30:00,07:45:00,08:00:00,08:15:00,08:30:00,08:45:00,09:00:00,09:15:00,09:30:00,09:45:00,10:00:00,10:15:00,10:30:00,10:45:00,11:00:00,11:15:00,11:30:00,11:45:00,12:00:00,12:15:00,12:30:00,12:45:00,13:00:00,13:15:00,13:30:00,13:45:00,14:00:00,14:15:00,14:30:00,14:45:00,14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,2017-01-01 00:00:00,2017-01-01 06:00:00,2017-01-01 06:15:00,2017-01-01 06:30:00,2017-01-01 06:45:00,2017-01-01 07:00:00,2017-01-01 07:15:00,2017-01-01 07:30:00,2017-01-01 07:45:00,2017-01-01 08:00:00,2017-01-01 08:15:00,2017-01-01 08:30:00,2017-01-01 08:45:00,2017-01-01 09:00:00,2017-01-01 09:15:00,2017-01-01 09:30:00,2017-01-01 09:45:00,2017-01-01 10:00:00,2017-01-01 10:15:00,2017-01-01 10:30:00,2017-01-01 10:45:00,2017-01-01 11:00:00,2017-01-01 11:15:00,2017-01-01 11:30:00,2017-01-01 11:45:00,2017-01-01 12:00:00,2017-01-01 12:15:00,2017-01-01 12:30:00,2017-01-01 12:45:00,2017-01-01 13:00:00,2017-01-01 13:15:00,2017-01-01 13:30:00,2017-01-01 13:45:00,2017-01-01 14:00:00,2017-01-01 14:15:00,2017-01-01 14:30:00,2017-01-01 14:45:00,2017-01-01 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.113000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,SAINDO,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:

In [106]:
planilha1 = planilha1.fillna('')

In [107]:
pd.set_option('display.max_columns', None)

In [108]:
planilha1.head(50)

,Unnamed: 0,Unnamed: 1,TURNO MANHÃ,Unnamed: 3,6,Unnamed: 5,Unnamed: 6,Unnamed: 7,7,Unnamed: 9,Unnamed: 10,Unnamed: 11,8,Unnamed: 13,Unnamed: 14,Unnamed: 15,9,Unnamed: 17,Unnamed: 18,Unnamed: 19,10,Unnamed: 21,Unnamed: 22,Unnamed: 23,11,Unnamed: 25,Unnamed: 26,Unnamed: 27,12,Unnamed: 29,Unnamed: 30,Unnamed: 31,13,Unnamed: 33,Unnamed: 34,Unnamed: 35,14,Unnamed: 37,Unnamed: 38,Unnamed: 39,Horas,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,30,45,0,15,30.1,45.1,0.1,15.1,30.2,45.2,0.2,15.2,30.3,45.3,0.3,15.3,30.4,45.4,0.4,15.4,30.5,45.5,0.5,15.5,30.6,45.6,0.6,15.6,30.7,45.7,0.7,15.7,30.8,45.8,0.8,15.8,30.9,45.9
0,,,NOME,,0,15,30,45,0,15,30,45,0,15,30,45,0,15,30,45,0,15,30,45,0,15,30,45,0,15,30,45,0,15,30,45,0,15,30,45,Trabalhadas,,,,,,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,,,,,06:00:00,06:15:00,06:30:00,06:45:00,07:00:00,07:15:00,07:30:00,07:45:00,08:00:00,08:15:00,08:30:00,08:45:00,09:00:00,09:15:00,09:30:00,09:45:00,10:00:00,10:15:00,10:30:00,10:45:00,11:00:00,11:15:00,11:30:00,11:45:00,12:00:00,12:15:00,12:30:00,12:45:00,13:00:00,13:15:00,13:30:00,13:45:00,14:00:00,14:15:00,14:30:00,14:45:00,14:00:00,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,,,,2017-01-01 00:00:00,2017-01-01 06:00:00,2017-01-01 06:15:00,2017-01-01 06:30:00,2017-01-01 06:45:00,2017-01-01 07:00:00,2017-01-01 07:15:00,2017-01-01 07:30:00,2017-01-01 07:45:00,2017-01-01 08:00:00,2017-01-01 08:15:00,2017-01-01 08:30:00,2017-01-01 08:45:00,2017-01-01 09:00:00,2017-01-01 09:15:00,2017-01-01 09:30:00,2017-01-01 09:45:00,2017-01-01 10:00:00,2017-01-01 10:15:00,2017-01-01 10:30:00,2017-01-01 10:45:00,2017-01-01 11:00:00,2017-01-01 11:15:00,2017-01-01 11:30:00,2017-01-01 11:45:00,2017-01-01 12:00:00,2017-01-01 12:15:00,2017-01-01 12:30:00,2017-01-01 12:45:00,2017-01-01 13:00:00,2017-01-01 13:15:00,2017-01-01 13:30:00,2017-01-01 13:45:00,2017-01-01 14:00:00,2017-01-01 14:15:00,2017-01-01 14:30:00,2017-01-01 14:45:00,2017-01-01 14:00:00,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,,,,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.081000,2017-01-01 11:50:44.113000,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,,,,SAINDO,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5,,,,CHEGANDO,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6,,,,AGORA,00:00:00,00:00:00,00:

In [109]:
# Lê a planilha e carrega em um dataframe, limpando os dados NaN
planilha1 = pd.read_excel('/content/HorariosTeste/2022_2023/Horário RSP - Backup - 12-04-2023 - WED.xlsx')
planilha1 = planilha1.fillna('')
planilha1.head(50)

,DATA,Unnamed: 1,Unnamed: 2,REGIÃO SP - MANHÃ,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,SUPERVISOR,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,ÁREA RESTRITA PARA OS DESENVOLVEDORES,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78
0,13/04/2023 - QUINTA-FEIRA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,HORA ATUAL,,,6,,,,7,,,,8,,,,9,,,,10,,,,11,,,,12,,,,13,,,,14,,,,15,,,,16,,,,,,,,,suporteplanilhasc1@gmail.com,,,,,,,,,,,,,,,,,,,,,,,,,,
2,2023-04-13 04:26:17.088000,,,0,15,30,45,0,15,30,45,0,15,30,45,0,15,30,45,0,15,30,45,0,15,30,45,0,15,30,45,0,15,30,45,0,15,30,45,0,15,30,45,0,15,30,45,2S NILSILENE (I) - SUPERVISOR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,,,,06:00,06:15,06:30,06:45,07:00,07:15,07:30,07:45,08:00,08:15,08:30,08:45,09:00,09:15,09:30,09:45,10:00,10:15,10:30,10:45,11:00,11:15,11:30,11:45,12:00,12:15,12:30,12:45,13:00,13:15,13:30,13:45,14:00,14:15,14:30,14:45,15:00,15:15,15:30,15:45,16:00,16:15,16:30,16:45,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,,,2023-04-13 00:00:00,2023-04-13 06:00:00,2023-04-13 06:15:00.288000,2023-04-13 06:29:59.712000,2023-04-13 06:45:00,2023-04-13 07:00:00.288000,2023-04-13 07:14:59.712000,2023-04-13 07:30:00,2023-04-13 07:45:00.288000,2023-04-13 07:59:59.712000,2023-04-13 08:15:00,2023-04-13 08:30:00.288000,2023-04-13 08:44:59.712000,2023-04-13 09:00:00,2023-04-13 09:15:00.288000,2023-04-13 09:29:59.712000,2023-04-13 09:45:00,2023-04-13 10:00:00.288000,2023-04-13 10:14:59.712000,2023-04-13 10:30:00,2023-04-13 10:45:00.288000,2023-04-13 10:59:59.712000,2023-04-13 11:15:00,2023-04-13 11:30:00.288000,2023-04-13 11:44:59.712000,2023-04-13 12:00:00,2023-04-13 12:15:00.288000,2023-04-13 12:29:59.712000,2023-04-13 12:45:00,2023-04-13 13:00:00.288000,2023-04-13 13:14:59.712000,2023-04-13 13:30:00,2023-04-13 13:45:00.288000,2023-04-13 13:59:59.712000,2023-04-13 14:15:00,2023-04-13 14:30:00.288000,2023-04-13 14:44:59.712000,2023-04-13 15:00:00,2023-04-13 15:15:00.288000,2023-04-13 15:29:59.712000,2023-04-13 15:45:00,2023-04-13 16:00:00.288000,2023-04-13 16:14:59.712000,2023-04-13 16:30:00,2023-04-13 16:45:00.288000,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5,,,2023-04-13 04:26:17.088000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2

Encontrando a linha de horas na planilha

In [110]:
# Lê a planilha e carrega em um dataframe, limpando os dados NaN
planilha1 = pd.read_excel('/content/HorariosTeste/2022_2023/Horário RSP - Backup - 12-04-2023 - WED.xlsx')
planilha1 = planilha1.fillna('')

# Encontra o índice da linha que contém o valor 06:00
linha_hrs_indices = planilha1.index[planilha1.iloc[:, 3] == '06:00']

# Busca a linha índice 06:00 e mostra o resultado
linha_hrs = planilha1.loc[linha_hrs_indices[0]]
linha_hrs

DATA                      
Unnamed: 1                
Unnamed: 2                
REGIÃO SP - MANHÃ    06:00
Unnamed: 4           06:15
                     ...  
Unnamed: 74               
Unnamed: 75               
Unnamed: 76               
Unnamed: 77               
Unnamed: 78               
Name: 3, Length: 79, dtype: object

In [111]:
# Lê a planilha e carrega em um dataframe, limpando os dados NaN
planilha1 = pd.read_excel('/content/HorariosTeste/2022_2023/Horário RSP - Backup - 12-04-2023 - WED.xlsx')
planilha1 = planilha1.fillna('')

# Encontra o índice da segunda ocorrência da palavra "OPERADORES"
linha_operadores2 = planilha1.index[planilha1.iloc[:, 0] == 'OPERADORES'][1]

# Busca a linha índice correspondente à linha 40
linha_operadores = planilha1.iloc[linha_operadores2]

# Imprime a linha inteira correspondente à linha 40
linha_operadores

DATA                 OPERADORES
Unnamed: 1                     
Unnamed: 2                     
REGIÃO SP - MANHÃ             0
Unnamed: 4                    0
                        ...    
Unnamed: 74                    
Unnamed: 75                    
Unnamed: 76                    
Unnamed: 77                    
Unnamed: 78                    
Name: 38, Length: 79, dtype: object

In [112]:
# Concatena as colunas dos dataframes
linha_op_hrs = pd.concat([linha_operadores, linha_hrs], axis=1)

# Mostra o resultado
linha_op_hrs

# Cria uma nova coluna com o número índice
linha_op_hrs = linha_op_hrs.reset_index()

# Renomeia a coluna "index" para "número índice"
linha_op_hrs = linha_op_hrs.rename(columns={"index": "número índice"})

# Mostra o resultado
linha_op_hrs

,número índice,38,3
0,DATA,OPERADORES,
1,Unnamed: 1,,
2,Unnamed: 2,,
3,REGIÃO SP - MANHÃ,0,06:00
4,Unnamed: 4,0,06:15
...,...,...,...
74,Unnamed: 74,,
75,Unnamed: 75,,
76,Unnamed: 76,,
77,Unnamed: 77,,


In [113]:
# Renomeia as colunas
linha_op_hrs = linha_op_hrs.rename(columns={0: '', linha_op_hrs.columns[-1]: 'horas'})
linha_op_hrs = linha_op_hrs.rename({linha_op_hrs.columns[1]: formatted_date}, axis=1)

# Remove a primeira linha do dataframe
linha_op_hrs = linha_op_hrs.iloc[1:]

# Reseta o índice do dataframe
linha_op_hrs = linha_op_hrs.reset_index(drop=True)

# Deletando a primeira coluna do dataframe linha_op_hrs
linha_op_hrs = linha_op_hrs.drop(labels='número índice', axis=1)

# Mostra o resultado
linha_op_hrs

,12/04/2023,horas
0,,
1,,
2,0,06:00
3,0,06:15
4,0,06:30
...,...,...
73,,
74,,
75,,
76,,


In [114]:
# Limpando os valores nulos
linha_op_hrs = linha_op_hrs.query('horas != ""')

linha_op_hrs

,12/04/2023,horas
2,0,06:00
3,0,06:15
4,0,06:30
5,6,06:45
6,6,07:00
7,6,07:15
8,6,07:30
9,6,07:45
10,5,08:00
11,5,08:15


In [115]:
# Aplicação do transpose
linha_op_hrs_transpose = linha_op_hrs.transpose()
#Cria um arquivo csv para checar se está tudo ok
# linha_op_hrs_transpose.to_csv('linha_op_hrs_transpose.csv', index=False)
linha_op_hrs_transpose

,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45
12/04/2023,0,0,0,6,6,6,6,6,5,5,6,6,5,8,7,5,5,5,6,6,6,5,5,5,5,4,4,4,5,5,5,4,4,0,0,0,0,0,0,0,0,0,0,0
horas,06:00,06:15,06:30,06:45,07:00,07:15,07:30,07:45,08:00,08:15,08:30,08:45,09:00,09:15,09:30,09:45,10:00,10:15,10:30,10:45,11:00,11:15,11:30,11:45,12:00,12:15,12:30,12:45,13:00,13:15,13:30,13:45,14:00,14:15,14:30,14:45,15:00,15:15,15:30,15:45,16:00,16:15,16:30,16:45


# Fazendo um merge de df_test com linha_op_hrs_transpose

In [128]:
df.head(2)

,horas,num_traf,dd_mm_aaaa,ds_fs_ff,00:00,00:15,00:30,00:45,01:00,01:15,01:30,01:45,02:00,02:15,02:30,02:45,03:00,03:15,03:30,03:45,04:00,04:15,04:30,04:45,05:00,05:15,05:30,05:45,06:00,06:15,06:30,06:45,07:00,07:15,07:30,07:45,08:00,08:15,08:30,08:45,09:00,09:15,09:30,09:45,10:00,10:15,10:30,10:45,11:00,11:15,11:30,11:45,12:00,12:15,12:30,12:45,13:00,13:15,13:30,13:45,14:00,14:15,14:30,14:45,15:00,15:15,15:30,15:45,16:00,16:15,16:30,16:45,17:00,17:15,17:30,17:45,18:00,18:15,18:30,18:45,19:00,19:15,19:30,19:45,20:00,20:15,20:30,20:45,21:00,21:15,21:30,21:45,22:00,22:15,22:30,22:45,23:00,23:15,23:30,23:45
0,D1Y1,1048,01/01/2017,DOM,4,2,3,2,2,2,2,2,2,2,2,2,2,7,4,10,7,10,4,10,9,8,9,8,2,3,7,7,7,7,5,4,7,10,9,3,7,2,9,2,6,10,3,2,9,7,5,6,3,9,5,4,7,5,9,6,10,10,9,2,9,7,4,3,10,7,4,10,9,4,10,9,5,2,10,5,10,5,4,2,9,7,7,6,2,5,6,2,9,4,9,8,4,4,4,6
1,D2Y1,962,02/01/2017,SEG,3,4,5,2,2,2,2,2,2,2,2,2,2,9,7,8,10,8,8,4,2,4,4,7,2,6,8,9,8,6,8,7,7,8,5,8,7,7,4,10,2,6,4,5,3,6,3,2,7,7,5,8,4,3,5,8,7,6,10,9,10,9,10,6,4,10,5,10,4,9,2,4,5,5,3,10,9,4,2,10,6,6,3,2,5,2,9,9,8,10,9,9,4,3,3,10


In [129]:
# Montando um df_test para não utilizar o df original
df_test = df.head(2)

# Excluindo colunas 0 a 2 , 3 a 28 e 73 a 100 do df_test
# dd_mm_aaaa (06:00 ás 16:45)
cols_to_remove = np.r_[0:2, 3:28, 72:100]
df_test = df_test.drop(df_test.columns[cols_to_remove], axis=1)

# Transpondo
df_test = df_test.transpose()



# Mudando o index das colunas
# Definindo a primeira linha como rótulos das colunas
df_test = df_test.set_axis(df_test.iloc[0], axis=1)

# Excluindo a primeira linha
df_test = df_test.iloc[1:]

# Mostrando o resultado
df_test

dd_mm_aaaa,01/01/2017,02/01/2017
06:00,2,2
06:15,3,6
06:30,7,8
06:45,7,9
07:00,7,8
07:15,7,6
07:30,5,8
07:45,4,7
08:00,7,7
08:15,10,8


In [130]:
# Mostrando a df linha_op_hrs

linha_op_hrs.head()

,12/04/2023,horas
2,0,06:00
3,0,06:15
4,0,06:30
5,6,06:45
6,6,07:00


In [135]:
# Criando a variável col_name para resgatar o valor da linha 0 de linha_op_hrs
# para colocar esse valor col_name como rótulo da coluna em df_test
col_name = linha_op_hrs.columns[0]

# Inserindo a coluna 0 de linha_op_hrs na posição 2 em df_test
df_test.insert(loc=1, column=col_name, value=linha_op_hrs.iloc[:, 0])

df_test.head()

ValueError: ignored

In [124]:
df_test.transpose().head()

,06:00,06:15,06:30,06:45,07:00,07:15,07:30,07:45,08:00,08:15,08:30,08:45,09:00,09:15,09:30,09:45,10:00,10:15,10:30,10:45,11:00,11:15,11:30,11:45,12:00,12:15,12:30,12:45,13:00,13:15,13:30,13:45,14:00,14:15,14:30,14:45,15:00,15:15,15:30,15:45,16:00,16:15,16:30,16:45
dd_mm_aaaa,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
01/01/2017,2,3,7,7,7,7,5,4,7,10,9,3,7,2,9,2,6,10,3,2,9,7,5,6,3,9,5,4,7,5,9,6,10,10,9,2,9,7,4,3,10,7,4,10
12/04/2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
02/01/2017,2,6,8,9,8,6,8,7,7,8,5,8,7,7,4,10,2,6,4,5,3,6,3,2,7,7,5,8,4,3,5,8,7,6,10,9,10,9,10,6,4,10,5,10


# Prompts que deram certo no chatgpt
- Mostre uma solução totalmente alternativa para esse código, pois não está dando certo de maneira alguma, você está em um looping infinito, monte um código totalmetne novo e saia já desse looping errôneo (Pt-br)


- Preciso criar uma PDF AI Line Numbers identificator, ou seja, uma IA que identifica uma sequencia de números dentro de um PDF, dentre várias palavras, símbolos, códigos, existem sequencias de números que podem estar separados por vírgulas, traços, espaços, etc. Dessa forma a IA identificará a sequencia e no final dará um print da linha com a sequencia identificada dentro do PDF.

Crie uma solução inovadora para eu montar utilizando o google colab e bibliotecas de redes neurais profundas (deep learning).

Siga as etapas para construção da Machine Learning:
# 1. Importação de bibliotecas

# 2. Dados de entrada para o treinamento

# 2.1 Criação de 50 sequências aleatórias de 50 números de 2 a 15 (inclusive)

# 2.2 Conversão das sequências em matrizes para treinamento do modelo

# 3. Arquitetura do modelo

# 4. Redimensionamento das dimensões dos dados de entrada para interpretação do modelo

# 5. Criação do modelo

# 6. Treinamento o modelo

# 7. Teste do modelo com a frase ("Olá mundo! Quero identificar a seguinte sequência de números
2 4  6  7 8  8  9  2   3 .  2, 5; 3-8´2~7=8 5 5 2)


In [137]:
import os

# Diretório onde os arquivos estão localizados
diretorio = '/content/HorariosTeste'

# Dicionário para armazenar os arquivos por extensão
arquivos_por_extensao = {}

# Percorre todos os arquivos em todas as subpastas do diretório
for pasta_atual, subpastas, arquivos in os.walk(diretorio):
    for nome_arquivo in arquivos:
        # Obtém a extensão do arquivo
        extensao = os.path.splitext(nome_arquivo)[1]
        # Adiciona o arquivo ao dicionário de acordo com a extensão
        if extensao in arquivos_por_extensao:
            arquivos_por_extensao[extensao].append(os.path.join(pasta_atual, nome_arquivo))
        else:
            arquivos_por_extensao[extensao] = [os.path.join(pasta_atual, nome_arquivo)]

# Imprime os arquivos por extensão
for extensao, arquivos in arquivos_por_extensao.items():
    print(extensao + ':')
    for arquivo in arquivos:
        print('\t' + arquivo)


A saída de streaming foi truncada nas últimas 5000 linhas.
	/content/HorariosTeste/2019/01 JANEIRO/PERNOITE/03 JAN 2019 QUI PERNOITE.xls
	/content/HorariosTeste/2019/01 JANEIRO/PERNOITE/09 JAN 2019 QUA PERNOITE.xls
	/content/HorariosTeste/2019/01 JANEIRO/PERNOITE/14 JAN 2019 SEG PERNOITE.xls
	/content/HorariosTeste/2019/01 JANEIRO/PERNOITE/30 JAN 2019 QUA PERNOITE.xls
	/content/HorariosTeste/2019/01 JANEIRO/PERNOITE/28 JAN 2019 SEG PERNOITE.xls
	/content/HorariosTeste/2019/01 JANEIRO/PERNOITE/17 JAN 2019 QUI PERNOITE.xls
	/content/HorariosTeste/2019/01 JANEIRO/PERNOITE/22 JAN 2019 TER PERNOITE.xls
	/content/HorariosTeste/2019/01 JANEIRO/PERNOITE/15 JAN 2019 TER PERNOITE.xls
	/content/HorariosTeste/2019/01 JANEIRO/PERNOITE/23 JAN 2019 QUA PERNOITE.xls
	/content/HorariosTeste/2019/01 JANEIRO/PERNOITE/12 JAN 2019 SAB PERNOITE.xls
	/content/HorariosTeste/2019/01 JANEIRO/PERNOITE/21 JAN 2019 SEG PERNOITE.xls
	/content/HorariosTeste/2019/01 JANEIRO/PERNOITE/25 JAN 2019 QUI PERNOITE.xls
	/con